In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
#import pandas_profiling
%matplotlib inline

In [ ]:
train_ = pd.read_csv('./data/train.csv',index_col=0)
test_ = pd. read_csv('./data/test.csv',index_col=0)
sample_submission = pd.read_csv('./data/sample_submission.csv', index_col=0)

데이터 설명

-     대전지역에서 측정한 실내외 19곳의 센서데이터와, 주변 지역의 기상청 공공데이터를 semi-비식별화하여 제공합니다. 
-     센서는 온도를 측정하였습니다. 
-     모든 데이터는 시간 순으로 정렬 되어 있으며 10분 단위 데이터 입니다. 
-     예측 대상(target variable)은 Y18입니다. 

train.csv 

-     30일 간의 기상청 데이터 (X00 ~ X39) 및 센서데이터 (Y00~Y17)
-     이후 3일 간의 기상청 데이터 (X00~X39) 및 센서데이터 (Y18)

test.csv 
-     train.csv 기간 이후 80일 간의 기상청 데이터 (X00~X39)

sample_submission.csv
-     제출 양식 예시

X00 X07 X28 X31 기온  
X01 X06 X22 X27 기압  
X02 X03 X18 X24 풍속  
X04 X10 X21 X36 누적강수량  
X05 X08 X09 X23 해면기압  
X11 X14 X16 X19 누적일사량  
X12, X20, X30, X37 습도
X13 X15 X17 X25 풍향  

Y00~17 센서측정온도  

In [ ]:
train=train_.copy()
test=test_.copy()

### Y00 ~ Y17 >>> [0:4320]: Value, [4321:]: NaN, Y_index: [41:58]
### Y18 >>> [0:4320]: NaN, [4321:]: Value, Y_index: [59]

In [4]:
tp=0
k=0

In [5]:
cols=train.columns

In [6]:
tp=0
k=0
for i, col in enumerate(cols):
    if i > 39 and i <58:
        tp += train.iloc[0:4320,i]
        k+=1   


In [7]:
tmp=tp/k

In [8]:
train.iloc[0:4320,58]=tmp

In [30]:
for i in range(40):
    print(train.iloc[:,i].head())

id
0    9.7
1    9.3
2    9.4
3    9.4
4    9.2
Name: X00, dtype: float64
id
0    988.8
1    988.9
2    989.0
3    988.9
4    988.9
Name: X01, dtype: float64
id
0    1.2
1    1.7
2    1.1
3    1.5
4    0.8
Name: X02, dtype: float64
id
0    0.6
1    1.9
2    2.3
3    0.7
4    1.7
Name: X03, dtype: float64
id
0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: X04, dtype: float64
id
0    1009.3
1    1009.3
2    1009.2
3    1009.2
4    1009.2
Name: X05, dtype: float64
id
0    989.6
1    989.6
2    989.7
3    989.6
4    989.7
Name: X06, dtype: float64
id
0    12.2
1    12.1
2    12.1
3    12.0
4    12.0
Name: X07, dtype: float64
id
0    1009.9
1    1010.0
2    1010.1
3    1010.0
4    1010.1
Name: X08, dtype: float64
id
0    1009.8
1    1009.9
2    1010.1
3    1010.0
4    1010.0
Name: X09, dtype: float64
id
0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: X10, dtype: float64
id
0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: X11, dtype: float64
id
0    82.4
1    81.2
2    86.1
3    87.7

In [10]:
x=train.iloc[:,:40]
y=train.iloc[:,-1]

In [11]:
test_x=test.iloc[:,:40]

In [12]:
# !pip install catboost

In [13]:
from sklearn.model_selection import train_test_split, KFold, RandomizedSearchCV
from sklearn.model_selection import StratifiedShuffleSplit, ShuffleSplit

x_train, x_test, y_train, y_test = train_test_split(x,y, train_size=0.8, shuffle=False) #random_state=0)

In [28]:
from sklearn.ensemble import RandomForestRegressor
import lightgbm as lgbm
import xgboost as xgb
import catboost as cb

from sklearn.metrics import r2_score, mean_squared_error

params={ 'max_depth':3, 'n_estimators':2500,  'learning_rate': 0.007, \
        'boost_from_average': False, 'subsample': 1, 'loss_function':'RMSE', 'scale_pos_weight': 10,
        'coal_sample_bytree':'0.75'}

clf= xgb.XGBRegressor(**params)
#clf = cb.CatBoostRegressor(**params)
model=clf.fit(x_train,y_train, verbose=100)

/home/lab21/.local/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[15:27:45] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [24]:
y_pred=model.predict(x_test)

In [25]:
print( r2_score(y_pred, y_test), mean_squared_error(y_pred, y_test))

0.8479685270922396 4.3727977580103055


In [26]:
pred=model.predict(test_x)

In [27]:
submission = pd.DataFrame(data=pred, columns=sample_submission.columns, index=sample_submission.index)
submission.to_csv('submission.csv', index=True)